## Gene sampling for expression data comparisons

For mutation prediction from gene expression data, we want to compare the Vogelstein et al. 2013 gene set [located here](https://github.com/greenelab/pancancer/blob/master/data/vogelstein_cancergenes.tsv) with a set of random genes of equal length, and with a set of the most frequently mutated genes in TCGA of equal length.

This script will sample the random genes and identify the most frequently mutated genes, and generate files containing those genes and their classification information (oncogene/TSG/neither).

In [1]:
import os

import numpy as np
import pandas as pd

import mpmp.config as cfg
import mpmp.utilities.data_utilities as du

In [2]:
# this is the number of valid genes in the Vogelstein gene set
NUM_GENES = 107

# sample random genes from set of genes with every gene with >= NUM_CANCERS
# valid cancer types
#
# if we sampled them randomly from all genes, it's likely that many of them
# would end up with no valid cancer types (i.e. not enough mutations to train
# a classifier), so we add this criterion to make sure they're reasonably
# frequently mutated
NUM_CANCERS = 2

### Load mutation and sample/cancer type info

In [3]:
sample_info_df = du.load_sample_info('expression', verbose=True)
mutation_df = du.load_pancancer_data(verbose=True)[1]
print(sample_info_df.shape)
print(mutation_df.shape)

Loading sample info...
Loading pan-cancer data from cached pickle file...


(11060, 3)
(9074, 20938)


In [4]:
mutations_df = (mutation_df
    .merge(sample_info_df, how='inner', left_index=True, right_index=True)
    .drop(columns=['sample_type', 'id_for_stratification'])
)
print(mutations_df.shape)

### Get number of mutations per gene, per cancer type

In [5]:
sum_df = mutations_df.groupby('cancer_type').agg('sum')
count_df = mutations_df.groupby('cancer_type').agg('count')
ratio_df = sum_df / count_df
sum_df.iloc[:5, :5]

,5S_rRNA,A1BG,A1CF,A2M,A2ML1
cancer_type,,,,,
ACC,0,0,0,0,0
BLCA,0,4,3,15,12
BRCA,0,3,8,12,10
CESC,0,1,4,5,6
CHOL,0,0,1,1,0


In [6]:
SUM_THRESHOLD = 10
PROP_THRESHOLD = 0.1

sum_df = (sum_df > SUM_THRESHOLD)
ratio_df = (ratio_df > PROP_THRESHOLD)
valid_df = sum_df & ratio_df

print(sum_df.sum().sum())
print(ratio_df.sum().sum())
valid_df.iloc[:5, :5]

40436
2798


,5S_rRNA,A1BG,A1CF,A2M,A2ML1
cancer_type,,,,,
ACC,False,False,False,False,False
BLCA,False,False,False,False,False
BRCA,False,False,False,False,False
CESC,False,False,False,False,False
CHOL,False,False,False,False,False


### Sample randomly from set of all valid genes

In [8]:
valid_genes = valid_df.sum()[valid_df.sum() >= NUM_CANCERS]
print(valid_genes.head(10))
print(len(valid_genes))

ABCA12    3
ABCA13    8
ABCA4     2
ABCA8     2
ABCB1     2
ABCB5     2
ABCC12    2
ABCC9     2
ACACA     2
ACACB     2
dtype: int64
555


In [9]:
# sample randomly from valid genes and write to dataframe
sampled_genes = valid_genes.sample(n=NUM_GENES, random_state=cfg.default_seed)
print(sampled_genes.head())

HECTD4     3
PDE4DIP    5
BRAF       3
PKD1L1     2
CACNA1G    2
dtype: int64


In [11]:
# get oncogene/TSG status from Vogelstein gene list
# this is just used to decide whether to add copy number gains/losses in mutation labeling
vogelstein_df = du.load_vogelstein()
gene_to_class_map = dict(zip(vogelstein_df.gene, vogelstein_df.classification))

def get_class(gene):
    # if genes aren't in other gene lists, mark as 'neither'
    try:
        return gene_to_class_map[gene]
    except KeyError:
        return 'neither'
    
random_classes = [get_class(gene) for gene in sampled_genes.index.values]

random_df = pd.DataFrame({
    'gene': sampled_genes.index.values,
    'classification': random_classes
}).set_index('gene')

random_df.head()

,classification
gene,
HECTD4,neither
PDE4DIP,neither
BRAF,Oncogene
PKD1L1,neither
CACNA1G,neither


In [12]:
random_df.to_csv(cfg.random_genes, sep='\t')

### Get top mutated genes

Same methods as in https://github.com/greenelab/BioBombe/blob/master/9.tcga-classify/top-50-pancanatlas-mutations.ipynb (but we want more than 50 genes, since we want a gene set of the same size as Vogelstein)

In [13]:
mutation_count_df = mutation_df.sum().sort_values(ascending=False)
mutation_count_df.head()

TP53      3375
TTN       2841
MUC16     1815
PIK3CA    1290
CSMD3     1238
dtype: int64

In [14]:
top_genes = mutation_count_df[:NUM_GENES]
top_classes = [get_class(gene) for gene in top_genes.index.values]
top_df = pd.DataFrame({
    'gene': top_genes.index.values,
    'classification': top_classes
}).set_index('gene')
top_df.head()

,classification
gene,
TP53,TSG
TTN,neither
MUC16,neither
PIK3CA,Oncogene
CSMD3,neither


In [15]:
top_df.to_csv(cfg.top_genes, sep='\t')